# Chapter 1. The Python Data Model

The Python interpreter invokes special methods to perform basic object operations, often triggered by special syntax. The special method names are always written with leading and trailing double underscores (i.e., __getitem__). For example, the syntax `obj[key]` is supported by the `__getitem__` special method. In order to evaluate `my_collection[key]`, the interpreter calls `my_collection.__getitem__(key)`.

The special method names allow your objects to implement, support, and interact with basic language constructs such as:
- Iteration
- Collections
- Attribute access
- Operator overloading
- Function and method invocation
- Object creation and destruction
- String representation and formatting
- Managed contexts (i.e., `with` blocks)

### A Pythonic Card Deck

The following is a simple example demonstrating the power of two special methods, `__getitem__` and `__len__`.

**Example 1-1** is a class to represent a deck of playing cards.

*Example 1-1. A deck as a sequence of cards*

In [1]:
import collections

Card = collections.namedtuple('Card',['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
deck = FrenchDeck()
len(deck)

52

Reading specific cards from the deck - say the first or the last - should be as easy as `deck[0]` or `deck[-1]`, and this is what the `__getitem__` method provides:

In [4]:
deck[0]

Card(rank='2', suit='spades')

In [5]:
deck[-1]

Card(rank='A', suit='hearts')

Python already has a function we can use to get a random item from a sequence: `random.choice`. We can just use it on a deck instance:

In [6]:
from random import choice
choice(deck)

Card(rank='K', suit='diamonds')

In [7]:
choice(deck)

Card(rank='K', suit='clubs')

In [8]:
choice(deck)

Card(rank='J', suit='hearts')

So far, two advantages of using special methods to leverage the Python data model:
- Users don't have to memorize arbitrary method names for standard operations
- It's easier to use the Python standard library than having to reinvent the wheel

Because `__getitem__` delegates to the `[]` operator of `self._cards`, our deck automatically supports slicing. Here's how to pick the top three cards from a brand new deck, and then pick just the aces by starting on index 12 and skipping 13 cards at a time:

In [9]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [10]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

By implementing the `__getitem__` special method, our deck is also iterable:

In [11]:
for card in deck[:10]:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')


The deck can also be iterated in reverse:

In [12]:
for card in reversed(deck[:10]):
    print(card)

Card(rank='J', suit='spades')
Card(rank='10', suit='spades')
Card(rank='9', suit='spades')
Card(rank='8', suit='spades')
Card(rank='7', suit='spades')
Card(rank='6', suit='spades')
Card(rank='5', suit='spades')
Card(rank='4', suit='spades')
Card(rank='3', suit='spades')
Card(rank='2', suit='spades')


Iteration is often implicit. If a collection has no `__containts__` method, the `in` operator does a sequential scan. Case in point: `in` works without our `FrenchDeck` because it is iterable:

In [14]:
Card('Q', 'hearts') in deck

True

In [15]:
Card('7', 'beasts') in deck

False

How about sorting? A common system of ranking cards is by rank, then by suit, then spades/hearts/diamonds/clubs. Here is a function that ranks cards by that rule, returning 0 for the 2 of clubs and 51 for the ace of spades:

In [16]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

Given `spades_high`, we can now list our deck in order of increasing rank:

In [17]:
for card in sorted(deck, key=spades_high)[:10]:
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')


Although `FrenchDeck` implicitly inherits from `object`, its functionality is not inherited, but comes from leveraging the data model and composition. By implementing the special methods `__len__` and `__getitem__`, our `FrenchDeck` behaves like a standard Python sequence, allowing it to benefit from core language features (e.g., iteration and slicing) and from the standard library, as shown by the examples using `random.choice`,`reversed`, and `sorted`. Thanks to composition, `__len__` and `__getitem` implementations can hand off all the work to a `list` object, `self._cards`.

So far, a `FrenchDeck` cannot be shuffled, because it is *immutable*: the cards and their positions cannot be changed, except by violating encapsulation and handling the `_cards` attribute directly. In Chapter 11, that will be fixed by adding a one-line `__setitem__` method.

### How Special Methods are Used

Special methods are meant to be called by the Python interpreter, and not by you. You don't write `my_object.__len__()`. You write `len(my_object)` and, if `my_object` is an instance of a user-defined class, then Python calls the `__len__` instance method you implemented.

But for built-in types like `list`, `str`, `bytearray`, and so on, the interpreter takes a shortcut: the CPython implementation of `len()` actually returns the value of the `ob_size` field in the `PyVarObject` C struct that represents any variable-sized built-in object in memory. This is much faster than calling a method.

More often than not, the special method call is implicit. `for i in x:` actually causes the invocation of `iter(x)`, which in turn may call `x.__iter__()` if that is available.

Your code should not typically have many direct calls to special methods. Unless you are doing a lot of metaprogramming, you should be implementing special methods more often that invoking them explicitly.

The only special method that is frequently called by user code directly is `__init__`, to invoke the initializer of the superclass in your own `__init__` implementation.

If you need to invoke a special method, it is usualy better to call the related built-in function (e.g., len, iter, str, etc). These built-ins call the corresponding special method, but often provide other services, and - for built-in types - are faster than method calls.

Avoid creating arbitrary, custom attributes with the `__foo__` syntax because such names may acquire special meanings in the future, even if they are unused today.

### Emulating Numeric Types

Several special methods allow user objects to respond to operators such as `+`. We will implement a class to represent two-dimensional vectors (Euclidean vectors like those used in math and physics).

```
>>> v1 = Vector(2,4)
>>> v2 = Vector(2,1)
>>> v1 + v2
Vector(4, 5)
```
Note how the `+` operator produces a `Vector` result, which is displayed in a friendly manner in the console.

The `abs` built in function returns the absolute value of integers and floats, and the magnitude of `complex` numbers, so to be consistent, our API also uses `abs` to calculate the magnitude of a vector:
```
>>> v = Vector(3,4)
>>> abs(v)
5.0
```

We can also implement the `*` operator to perform scalar multiplication (i.e., multiplying a vector by a number to produce a new vector with the same direction and a multiplied magnitude):
```
>>> v * 3
Vector(9, 12)
>>> abs(v * 3)
15.0
```

**Example 1-2** is a `Vector` class implementing the operations just described, through the use of the special methods `__repr__`,`__abs__`,`__add__`, and `__mul__`.

*Example 1-2. A simple two-dimensional vector class*

In [18]:
from math import hypot

class Vector:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

### String Representation

The `__repr__` special method is called by the `repr` built-in to get the string representation of the object for inspection. If we did not implement `__repr__`, vector instances would be shown in the console like `<Vector object at 0x10e100070>`.

The interactive console and debugger call `repr` on the results of the expressions evaluated, as does the `%r` placeholder in classic formatting with the % operator, and the `!r` conversion field in the new Format String Syntax used in the `str.format` method.

Note that in our `__repr__` implementation, we used `%r` to obtain the standard representation of the attributes to be displayed. This is good practice, because it shows the crucial difference between `Vector(1, 2)` and `Vector('1', '2')` - the latter would not work in the context of this example, because the constructor's arguments must be numbers, not `str`.

The strings returned by `__repr__` should be unambiguous and, if possible, match the source code necessary to re-create the object being represented. That is why our chosen representation looks like calling the constructor of the class (e.g., `Vector(3, 4)`).

Contrast `__repr__` with `__str__`, which is called by the `str()` constructor and implicitly used by the `print` function. `__str__` should return a string suitable for display to end users.

If you only implement one of these special methods, choose `__repr__`, because when no custom `__str__` is available, Python will call `__repr__` as a fallback.

### Arithmetic Operators

Example 1-2 implements two operators: `+` and `*`, to show basic usage of `__add__` and `__mul__`. Note that in both cases, the methods create and return a new instance of `Vector`, and do not modify either operand - `self` or `other` are merely read. This is the expected behavior of infix operators: to create new objects and not touch their operands.

### Boolean Value of a Custom Type

Although Python has a `bool` type, it accepts any object in a boolean context, such as the expression controlling an `if` or `while` statement, or as operands to `and`, `or`, and `not`. To determine whether a value `x` is *truthy* or *falsy*, Python applies `bool(x)`, which always returns `True` or `False`. 

By default, instances of user-defined classes are considered truthy, unless either `__bool__` or `__len__` is implemented. Basically, `bool(x)` calls `x.__bool__()` and uses the result. If `__bool__` is not implemented, Python tries to invoke `x.__len__()`, and if that returns zero, `bool` returns `False`. Otherwise `bool` returns `True`.

Note how special method `__bool__` allows your objects to be consistent with the truth value testing rules defined in the "Built-in Types" chapter of *The Python Standard Library* documentation.

> ### NOTE 
> A faster implementation of `Vector.__bool__` is this:
> ```
> def __bool__(self):
    return bool(self.x or self.y)
> ```
> This is harder to read, but avoids the trip through `abs`, `__abs__`, the squares and square root. The explicit conversion to `bool` is needed because `__bool__` must return a boolean and `or` returns either operand as is: `x or y` evaluates to `x` if that is *truthy*, otherwise the result is `y`, whatever that is.

### Overview of Special Methods

The "Data Model" chapter of *The Python Language Reference* lists 83 special method names, 47 of which are used to implement aritchmetic, bitwise, and comparison operators.

As an overview of what is available, see Tables 1-1 and 1-2.

*Table 1-1. Special method names (operators excluded)*

| Category | Method Names|
| :------: | :---------: |
| String/bytes representation | `__repr__`, `__str__`, `__format__`, `__bytes__` |
| Conversion to number | `__abs__`, `__bool__`, `__complex__`, `__int__`, `__float__`, `__hash__`, `__index__` |
| Emulating collections | `__len__`, `__getitem__`, `__setitem__`, `__delitem__`, `__contains__` |
| Iteration | `__iter__`, `__reversed__`, `__next__` |
| Emulating Callables | `__call__` |
| Context Management | `__enter__`, `__exit__` |
| Instance Creation and Destruction | `__new__`, `__init__`, `__del__` |
| Attribute Management | `__getattr__`, `__getattribute__`, `__setattr__`, `__delattr__`, `__dir__` |
| Attribute Descriptors | `__get__`, `__set__`, `__delete__` |
| Class Services | `__prepare__`, `__instancecheck__`, `__subclasscheck__` |



*Table 1-2. Special method names for operators*

| Category | Method names and related operators |
| -------- | ---------------------------------- |
| Unary numeric operators | `__neg__-`, `__pos__+`, `__abs__abs()` |
| Rich | `__lt__<`, `__le__<=`, `__eq__==` |
| Comparison Operators | `__ne__!=`, `__gt__>`,`__ge__>=`|
| Arithmetic Operators | `__add__+`, `__sub__-`, `__mul__*`, `__truediv__/`, `__floordiv__//`, `__mod__%`, `__divmod__divmod()`, `__pow__** or pow()`, `__round__round()` |
| Reversed Arithmetic Operators | `__radd__`, `__rsub__`, `__rmul__`, `__rtruediv__`, `__rfloordiv__`, `__rmod__`, `__rdivmod__`, `__rpow__` |
| Augmented Assignment Arithmetic Operators | `__iadd__`, `__isub__`, `__imul__`, `__itruediv__`, `__ifloordiv__`, `__imod__`, `__ipow__` |
| Bitwise Operators | `__invert__~`, `__lshift__<<`, `__rshift__>>`, `__and__&`, `__or__|`, `__xor__^` |
| Reversed Bitwise Operators | `__rlshift__`, `__rrshift__`, `__rand__`, `__rxor__`, `__ror__` |
| Augmented Assignment Bitwise Operators | `__ilshift__`, `__irshift__`, `__iand__`, `__ixor__`, `__ior__` |

> ### TIP
> The reversed operators are fallbacks used when operands are swapped (`b * a`) instead of `a * b`) while augmented assignments are shortcuts combining an infix operator with variable assignment (`a = a * b` becomes `a *= b`).

### Chapter Summary

By implementing special methods, your objects can behave like the built-in types, enabling the expressive coding style the community considers Pythonic. A basic requirement for a Python object is to provide usable string representations of itself, one used for debugging and logging, another for presentation to end users. That is why the special methods `__repr__` and `__str__` exist in the data model.

Emulating sequences is one of the most widely used applications of the special methods.

Thanks to operator overloading, Python offers a rich selection of numeric types, fromt he built-ins to `decimal.Decimal` and `fractions.Fraction`, all supporting infix arithmetic operators. Implementing operators, including reversed operators and augmented assignment, will be shown in Chapter 13 via enhancements of the `Vector` example.

# Chapter 2. An Array of Sequences

Most of the discussion in this chapter applies to sequences in general, from the familiar `list` to `str` and `bytes` types that are new in Python 3.

Specific topics on lists, tuples, arrays, and queues are also covered here, but the focus on Unicode strings and byte sequences is deferred to Chapter 4. Also, the idea here is to cover sequence types that are already in use.

### Overview of Built-In Sequences

The standard library offers a rich selection of sequence types implemented in C: 

*Container sequences*
> `list`, `tuples`, and `collections.deque` can hold items of different types.

*Flat sequences*
> `str`, `bytes`, `bytearray`, `memoryview`, and `array.array` hold items of one type

*Container sequences* hold references to the objects they contain, which may be of any type, while *flat sequences* physically store the value of each item within its own memory space, and not as distinct objects.

Thus, flat sequences are more compact, but they are limited to holding primitive values like characters, bytes, and numbers.

Another way of grouping sequence types is by mutability:

*Mutable sequences*
> `list`,`bytearray`,`array.array`, `collections.deque`, and `memoryview`

*Immutable sequences*
> `tuple`, `str`, and `bytes`


The most fundamental sequency type is the `list` - mutable and mixed-type.

Mastering list comprehensions opens the door to generator expressions, which - among other uses - can produce elements to fill up sequences of any type.

### List Comprehensions and Generator Expressions

A quick way to build a sequence is using a list comprehension (if the target is a `list`) or a generator expression (for all other kinds of sequences)

**Tip**: List comprehensions = *listcomps*, generator expressions = *genexps*

**List Comprehensions and Readability**

*Example 2-1. Build a list of Unicode codepoints from a string*

In [4]:
symbols = '$¢£¥€¤'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))
codes

[36, 162, 163, 165, 8364, 164]

*Example 2-2. Build a list of Unicode codepoints from a string, take two*

In [5]:
symbols = '$¢£¥€¤'
codes = [ord(symbol) for symbol in symbols]
codes

[36, 162, 163, 165, 8364, 164]

A `for` loops may be used to do lots of different things: scanning a sequence to count or pick items, computing aggregates (sums, averages), or any number of other processing tasks. A listcomp is meant to do one thing only: build a new list.

It is possible to abuse list comprehensions to write truly incomprehensible code. If you are not doing something with the produced list, you should not use that syntax. Also, keep it short. If the list comprehension spans more than two lines, it is probably best to break it apart or rewrite as a plain old `for` loop. Use your best judgement.

**Syntax Tip**: In Python code, line breaks are ignored inside pairs of [], {}, or (). So you can build multiline lines, listcomps, geneps, dictionaries and the like without using ugly \ line continuation escape.

List comprehensions build lists from sequences or any other iterable type by filtering and transforming items. The `filter` and `map` built-ins can be composed to do the same, but readability suffers.

### Listcomps Versus map and filter

Listcomps do everything the `map` and `filter` functions do, without the contortions of the functionally challenged Python `lambda`.

*Example 2-3. The same list built by a listcomp and a map/filter composition*

In [6]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [7]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

### Cartesian Products

Listcomps can generate lists from the Cartesian product of two or more iterables. The items that make up the cartesian product are tuples made from items from every input iterable. The resulting list has a length equal to the lengths of the input iterables multiplied.

For example, imagine you need to produce a list of T-shirts available in two colors and three sizes. Example 2-4 shows how to produce that list using a listcomp. The result has six items.

*Example 2-4. Cartesian product using a list comprehension*

In [8]:
colors = ['black', 'white']
sizes = ['S','M','L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [10]:
for color in colors:
    for size in sizes:
        print((color,size))

('black', 'S')
('black', 'M')
('black', 'L')
('white', 'S')
('white', 'M')
('white', 'L')


In [11]:
tshirts = [(color,size) for size in sizes
                        for color in colors]
tshirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

Listcomps are a one-trick pony: they build lists. To fill up other sequence types, a genexp is the way to go.

### Generator Expressions

To initialize tuples, arrays, and other types of sequences, you could also start from a listcomp, but a genexp saves memory because it yields items one by one using the iterator protocol instead of building a whole list just to feed another constructor.

Genexps use the same syntax as listcomps, but are enclosed in parentheses rather than brackets. Example 2-5 shows basic usage of genexps to build a tuple and an array.

*Example 2-5. Initializing a tuple and an array from a generator expression*

In [12]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [13]:
import array
array.array('I', (ord(symbol) for symbol in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

Example 2-6 uses a genexp with a Cartesian product to print out a roster of T-shirts of two colors in three sizes. In contrast with Example 2-4, here the six-item list of T-shirts is never built in memory: the generator expression feeds the `for` loop producing one item at a time. If the two lists used in the Cartesian product had 1,000 items each, using a generator expression would save the expense of building a list with a million items just to feed the `for` loop.

*Example 2-6. Cartesian product in a generator expression*

In [14]:
colors = ['black','white']
sizes = ['S','M','L']
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


### Tuples Are Not Just Immutable Lists

Some Python texts present tuples as "immutable lists", but that is short selling them. Tuples do double duty: they can be used as immutable lists and also as records with no field names. This use is sometimes overlooked.

**Tuples as Records**
Tuples hold records, each item in the tuple holds the data for one fields and the position of the item gives its meaning.

If you think of a tuple just as an immutable list, the quantity and the order of the items may or may not be important, the number of items is often fixed and their order is always vital.

Example 2-7 shows tuples being used as records. Note that in every expression, sorting the tuple would destroy the information because the meaning of each data item is given by its position in the tuple.

*Example 2-7. Tuples used as records*

In [ ]:
lax_coordinates = (39.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP','XDA205856')]
for passport in sorted(traveler_ids):
    print('%s/%s' % passport)